In [77]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error


In [78]:
#Already standardized data
path = "data/features/images_with_time_and_sensors_processed.csv"
df = pd.read_csv(path,sep=";")

In [79]:
df.head()

,set_id,time_dataset,type_adhesion,type_flank_wear,type_flank_wear+adhesion,wear,z0,z1,z2,z3,...,sens_emb_25,sens_emb_26,sens_emb_27,sens_emb_28,sens_emb_29,sens_emb_30,sens_emb_31,target_adhesion,target_flank,target_flank_adhesion
0,1,6,1,0,0,75.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.020842,0.0,0.116870,0.160032,0.0,45.0,0.0,0.0
1,1,8,1,0,0,45.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,-0.062780,0.0,0.079832,0.122237,0.0,60.0,0.0,0.0
2,1,9,1,0,0,60.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,-0.122946,0.0,0.032184,0.087577,0.0,60.0,0.0,0.0
3,1,11,1,0,0,60.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,-0.165228,0.0,0.047889,0.100234,0.0,75.0,0.0,0.0
4,1,27,0,1,0,90.0,0.0,0.0,0.0,0.856022,...,0.0,0.0,-0.100618,0.0,0.049117,0.102272,0.0,75.0,105.0,0.0


In [80]:
#Parameters in sets.csv
static_params_csv = "data/rawsets/sets.csv"
static_df = pd.read_csv(static_params_csv)

#Extract numeric set ID from the first column
static_df.rename(columns={static_df.columns[0]: "set_str"}, inplace=True)
static_df["set"] = static_df["set_str"].str.extract(r"Set (\d+)").astype(int)
static_df.set_index("set", inplace=True)

#Replace "?" with NaN
static_df.replace("?", np.nan, inplace=True)

#Convert numeric columns to float
numeric_cols = ["Vc"] #, "n", "fz", "Vf", "Ae", "Ap"
for col in numeric_cols:
    static_df[col] = pd.to_numeric(static_df[col], errors='coerce')

#Fill NaNs with median
static_df[numeric_cols] = static_df[numeric_cols].fillna(static_df[numeric_cols].median())

feature_cols = numeric_cols

static_df = static_df[feature_cols].reset_index()

static_df = static_df.rename(columns={"set": "set_id"})

#Only use cutting parameters that might be useful for sensor data, so exclude crop, Coating, z, material for now
#Might be worth it to include information of material, but for now we use only until set 13, which means one set with different material
print(static_df)

df = df.merge(static_df[["set_id", "Vc"]].rename(columns={"Vc": "Vc_raw"}),on="set_id", how="left")


    set_id     Vc
0        1  162.0
1        2  120.0
2        3  150.0
3        4  174.0
4        5  174.0
5        6  174.0
6        7  174.0
7        8  174.0
8        9  174.0
9       10  174.0
10      11  174.0
11      12  120.0
12      13  150.0
13      14  135.0
14      15  120.0
15      16  150.0
16      17  150.0


In [81]:
print(df[["set_id", "Vc_raw"]].drop_duplicates().sort_values("set_id").head(20))
print("Missing Vc_raw:", df["Vc_raw"].isna().sum())


      set_id  Vc_raw
0          1   162.0
25         2   120.0
121        3   150.0
224        4   174.0
321        5   174.0
382        6   174.0
529        7   174.0
629        8   174.0
728        9   174.0
828       10   174.0
928       11   174.0
1028      12   120.0
1078      13   150.0
1128      14   135.0
1280      15   120.0
1330      16   150.0
1426      17   150.0
Missing Vc_raw: 0


In [82]:
df.head()

,set_id,time_dataset,type_adhesion,type_flank_wear,type_flank_wear+adhesion,wear,z0,z1,z2,z3,...,sens_emb_26,sens_emb_27,sens_emb_28,sens_emb_29,sens_emb_30,sens_emb_31,target_adhesion,target_flank,target_flank_adhesion,Vc_raw
0,1,6,1,0,0,75.0,0.0,0.0,0.0,0.000000,...,0.0,0.020842,0.0,0.116870,0.160032,0.0,45.0,0.0,0.0,162.0
1,1,8,1,0,0,45.0,0.0,0.0,0.0,0.000000,...,0.0,-0.062780,0.0,0.079832,0.122237,0.0,60.0,0.0,0.0,162.0
2,1,9,1,0,0,60.0,0.0,0.0,0.0,0.000000,...,0.0,-0.122946,0.0,0.032184,0.087577,0.0,60.0,0.0,0.0,162.0
3,1,11,1,0,0,60.0,0.0,0.0,0.0,0.000000,...,0.0,-0.165228,0.0,0.047889,0.100234,0.0,75.0,0.0,0.0,162.0
4,1,27,0,1,0,90.0,0.0,0.0,0.0,0.856022,...,0.0,-0.100618,0.0,0.049117,0.102272,0.0,75.0,105.0,0.0,162.0


In [83]:
#Model

class PhysicsGuidedFusion(nn.Module):
    #Predicts T (total tool life)
    #Wear for three types (flank_wear, adhesion, flank+adhesion)
    
    def __init__(self, x_dim, wear_dim=3, n_taylor=0.3):
        super().__init__()
        self.n = float(n_taylor)

        #Learnable Taylor constant C, softplus keeps it positive, initial guess is 1
        self._C_raw = nn.Parameter(torch.tensor(1.0))  

        #Encoder for predicting T
        self.T_encoder = nn.Sequential(
            nn.Linear(x_dim,128),
            nn.ReLU(),
            nn.Linear(128,64),
            nn.ReLU()
        )
        #Head makes it a single scalar
        self.T_head = nn.Linear(64, 1)  

        #Predicts wear, tau is the extra feature (+ 1)
        self.wear_net = nn.Sequential(
            nn.Linear(x_dim +1,128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, wear_dim)
        )

    #The + 1e-6 makes C never zero
    @property
    def C(self):
        return F.softplus(self._C_raw) + 1e-6  

    #Forward pass, what happens during training
    def forward(self, x, t):
        
        #Predicting logT for numerical stability
        latent = self.T_encoder(x)
        logT = self.T_head(latent)
        
        #Normal T
        T_pred = torch.exp(logT)
        
        #Clamp to avoid extreme values, based on the typical number of samples per set (100)
        T_pred = torch.clamp(T_pred, min=50.0, max=150.0)
        
        #Recompute logT
        logT = torch.log(T_pred)

        #Calculates normalized tool life, how far is the tool along its total life, while avoiding division by 0
        tau = t / (T_pred + 1e-6)
        
        #Predict wear from both data and normalized tool life
        wear_in = torch.cat([x, tau], dim=1)
        wear_pred = self.wear_net(wear_in)

        #Return everything to check values later
        return wear_pred, T_pred, tau, logT


In [84]:
#

def taylor_physics_loss_log(Vc, logT, C, n):
    #Physics-based loss enforcing Taylor's tool life equation in log space for numerical stability
    #Equation: Vc * T^n = C
    #Taking the logarithm gives a linear form: log(Vc) + n * log(T) - log(C) = 0

    #Advoiding invalid values
    Vc = torch.clamp(Vc, min=1e-6)
    C  = torch.clamp(C, min=1e-6)

    res = torch.log(Vc) + n * logT - torch.log(C)
    
    #Mean squared physics residual
    return (res ** 2).mean()


In [85]:
y_cols = ['target_adhesion','target_flank', 'target_flank_adhesion']
t_col  = "time_dataset" 
Vc_col = "Vc_raw"

#Same drops as in Miguel's code
cols_to_drop = [
    'set_id', 'wear', 'wear_level', 'type',
    'sample_index_scaled',
    'target_adhesion', 'target_flank', 'target_flank_adhesion'
]

X_df = df.drop(columns=[c for c in cols_to_drop if c in df.columns], errors='ignore').copy()
y_df = df[y_cols].copy()

#NaNs
X_df = X_df.fillna(0)
y_df = y_df.fillna(0)

In [86]:
experiments = [
    {
        "name": "A. Paper Baseline",
        "desc": "Standard MATWI split",
        "test_sets": [4, 9, 13], #4, 9, 13
        "train_sets": [1, 2, 3, 5, 6, 7, 8, 10, 11, 12, 14, 15, 16, 17] #It includes also set 14 to 17, so not exactly their split
    }
    #,
    # {
    #     "name": "B. Unseen Material",
    #     "desc": "Train Std -> Test New Mat",
    #     "test_sets": [16, 17],
    #     "train_sets": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
    # },
    # {
    #     "name": "C. Complex Wear",
    #     "desc": "High Flank+Adhesion",
    #     "test_sets": [3, 12],
    #     "train_sets": [1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17]
    # }
]

In [87]:
#A. Paper Baseline
exp = experiments[0] 

#Split according to experiment
train_ids = exp["train_sets"]
test_ids  = exp["test_sets"]

train_mask = df["set_id"].isin(train_ids)
test_mask  = df["set_id"].isin(test_ids)

X_train_df = X_df[train_mask].copy()
y_train_df = y_df[train_mask].copy()

X_test_df = X_df[test_mask].copy()
y_test_df = y_df[test_mask].copy()

#Get t and Vc
t_train = X_train_df[[t_col]].to_numpy(dtype=np.float32)
Vc_train = X_train_df[[Vc_col]].to_numpy(dtype=np.float32)

t_test = X_test_df[[t_col]].to_numpy(dtype=np.float32)
Vc_test = X_test_df[[Vc_col]].to_numpy(dtype=np.float32)

#Remove t and Vc_raw from X features
X_train_df = X_train_df.drop(columns=[t_col])
X_test_df  = X_test_df.drop(columns=[t_col])

X_train_df = X_train_df.drop(columns=[Vc_col])
X_test_df  = X_test_df.drop(columns=[Vc_col])


In [88]:
X_train_df.head()

,type_adhesion,type_flank_wear,type_flank_wear+adhesion,z0,z1,z2,z3,z4,z5,z6,...,sens_emb_22,sens_emb_23,sens_emb_24,sens_emb_25,sens_emb_26,sens_emb_27,sens_emb_28,sens_emb_29,sens_emb_30,sens_emb_31
0,1,0,0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.247417,0.0,0.0,0.020842,0.0,0.116870,0.160032,0.0
1,1,0,0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.258990,0.0,0.0,-0.062780,0.0,0.079832,0.122237,0.0
2,1,0,0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.291091,0.0,0.0,-0.122946,0.0,0.032184,0.087577,0.0
3,1,0,0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.276417,0.0,0.0,-0.165228,0.0,0.047889,0.100234,0.0
4,0,1,0,0.0,0.0,0.0,0.856022,0.0,0.0,0.0,...,0.0,0.0,0.286767,0.0,0.0,-0.100618,0.0,0.049117,0.102272,0.0


In [89]:
#Creates validation set within training data
X_tr_df, X_val_df, y_tr_df, y_val_df, t_tr, t_val, Vc_tr, Vc_val = train_test_split(
    X_train_df, y_train_df, t_train, Vc_train,
    test_size=0.15,
    random_state=42
)

In [90]:
device = "cuda" if torch.cuda.is_available() else "cpu"

#Converting data to tensors, tr is train, t is tensor
#X
X_tr_t  = torch.tensor(X_tr_df.to_numpy(dtype=np.float32),dtype=torch.float32, device=device)
X_val_t = torch.tensor(X_val_df.to_numpy(dtype=np.float32),dtype=torch.float32, device=device)
X_test_t= torch.tensor(X_test_df.to_numpy(dtype=np.float32),dtype=torch.float32, device=device)

#y
y_tr_t  = torch.tensor(y_tr_df.to_numpy(dtype=np.float32),dtype=torch.float32, device=device)
y_val_t = torch.tensor(y_val_df.to_numpy(dtype=np.float32),dtype=torch.float32, device=device)
y_test_t= torch.tensor(y_test_df.to_numpy(dtype=np.float32),dtype=torch.float32, device=device)

#t
t_tr_t  = torch.tensor(t_tr, dtype=torch.float32, device=device)
t_val_t = torch.tensor(t_val,dtype=torch.float32, device=device)
t_test_t= torch.tensor(t_test,dtype=torch.float32, device=device)

#Vc
Vc_tr_t  = torch.tensor(Vc_tr,dtype=torch.float32, device=device)
Vc_val_t = torch.tensor(Vc_val,dtype=torch.float32,device=device)
Vc_test_t= torch.tensor(Vc_test,dtype=torch.float32, device=device)

In [91]:
model = PhysicsGuidedFusion(
    x_dim=X_tr_t.shape[1],
    wear_dim=y_tr_t.shape[1],
    n_taylor=0.3
).to(device)

#Give initial guess of T
with torch.no_grad():
    model.T_head.bias.fill_(np.log(100.0))

opt = torch.optim.Adam(model.parameters(), lr=1e-3)

#Strength of physics constraint
lambda_phys = 0.5    

batch_size  = 128
max_epochs  = 300
patience    = 20


In [96]:
model.eval()
with torch.no_grad():
    wear0, T0, tau0, logT0 = model(X_tr_t[:5], t_tr_t[:5])

print("Wear shape:", wear0.shape)
print("Initial logT:", logT0.squeeze().cpu().numpy())
print("Initial T_pred:", T0.squeeze().cpu().numpy())
print("Initial tau:", tau0.squeeze().cpu().numpy())
print("Initial C:", float(model.C.detach().cpu()))


Wear shape: torch.Size([5, 3])
Initial logT: [3.912023 3.912023 3.912023 3.912023 3.912023]
Initial T_pred: [50. 50. 50. 50. 50.]
Initial tau: [0.18 1.54 1.02 0.5  1.82]
Initial C: 3.302853584289551


In [93]:
#Training

#Keep track of best performance
best_val = float("inf")
best_state = None
bad = 0

#Number of training samples
N = X_tr_t.shape[0]

for epoch in range(1, max_epochs + 1):
    model.train()

    # shuffle indices
    perm = torch.randperm(N, device=device)

    #Mini-batch training
    for i in range(0, N, batch_size):
        idx = perm[i:i+batch_size]

        xb  = X_tr_t[idx]
        yb  = y_tr_t[idx]
        tb  = t_tr_t[idx]
        vcb = Vc_tr_t[idx]

        #Forward pass
        wear_pred, T_pred, tau, logT = model(xb, tb)

        data_loss = F.mse_loss(wear_pred, yb)
        phys_loss = taylor_physics_loss_log(vcb, logT, model.C, model.n)

        #Combined loss
        loss = data_loss + lambda_phys * phys_loss

        #Backpropagation and optimizer step
        opt.zero_grad()
        loss.backward()
        opt.step()

    #Validation MAE
    model.eval()
    with torch.no_grad():
        wear_val_hat, _, _, _ = model(X_val_t, t_val_t)
        val_mae = mean_absolute_error(
            y_val_t.cpu().numpy(),
            wear_val_hat.cpu().numpy()
        )

    if epoch == 1 or epoch % 10 == 0:
        print(f"Epoch {epoch:3d} | val_MAE={val_mae:.4f} | C={float(model.C.detach().cpu()):.4f}")

    #Early stopping
    if val_mae < best_val - 1e-6:
        best_val = val_mae
        best_state = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}
        bad = 0
    else:
        bad += 1
        if bad >= patience:
            print(f"Early stopping at epoch {epoch} (best val MAE={best_val:.4f})")
            break

#Get best model
if best_state is not None:
    model.load_state_dict(best_state)


Epoch   1 | val_MAE=48.7603 | C=1.3198
Epoch  10 | val_MAE=49.2357 | C=1.3791
Epoch  20 | val_MAE=42.3547 | C=1.4455
Epoch  30 | val_MAE=32.4588 | C=1.5123
Epoch  40 | val_MAE=26.3987 | C=1.5794
Epoch  50 | val_MAE=24.5096 | C=1.6467
Epoch  60 | val_MAE=23.2407 | C=1.7141
Epoch  70 | val_MAE=22.3979 | C=1.7816
Epoch  80 | val_MAE=21.8690 | C=1.8492
Epoch  90 | val_MAE=21.5510 | C=1.9169
Epoch 100 | val_MAE=20.9554 | C=1.9845
Epoch 110 | val_MAE=20.5240 | C=2.0521
Epoch 120 | val_MAE=20.0398 | C=2.1198
Epoch 130 | val_MAE=19.4952 | C=2.1874
Epoch 140 | val_MAE=19.1960 | C=2.2549
Epoch 150 | val_MAE=18.6754 | C=2.3224
Epoch 160 | val_MAE=18.2521 | C=2.3898
Epoch 170 | val_MAE=17.8530 | C=2.4572
Epoch 180 | val_MAE=17.4671 | C=2.5245
Epoch 190 | val_MAE=17.0886 | C=2.5917
Epoch 200 | val_MAE=16.5719 | C=2.6590
Epoch 210 | val_MAE=16.2295 | C=2.7262
Epoch 220 | val_MAE=15.7424 | C=2.7933
Epoch 230 | val_MAE=15.4429 | C=2.8604
Epoch 240 | val_MAE=15.0755 | C=2.9274
Epoch 250 | val_MAE=14.88

In [98]:
model.eval()
with torch.no_grad():
    wear_test_pred, T_test_pred, tau_test, _ = model(X_test_t, t_test_t)

y_pred = wear_test_pred.cpu().numpy()
y_true = y_test_t.cpu().numpy()

mae_global = mean_absolute_error(y_true, y_pred)
mae_per_col = mean_absolute_error(y_true, y_pred, multioutput="raw_values")

print("TEST RESULTS\n")
print("Global MAE:", mae_global)
print("MAE [adhesion, flank_wear, flank_wear+adhesion]:", mae_per_col)
print("Learned C:", float(model.C.detach().cpu()))
print("T_pred min mean max:", float(T_test_pred.min().cpu()), float(T_test_pred.mean().cpu()),float(T_test_pred.max().cpu()))
print("tau min mean max:", float(tau_test.min().cpu()), float(tau_test.mean().cpu()),float(tau_test.max().cpu()))


TEST RESULTS

Global MAE: 31.124330520629883
MAE [adhesion, flank_wear, flank_wear+adhesion]: [33.013664 50.68568   9.673655]
Learned C: 3.302853584289551
T_pred min mean max: 50.0 50.0 50.0
tau min mean max: 0.019999999552965164 0.9492307305335999 2.0799999237060547
